In [1]:
import random

import pandas as pd
import numpy as np

from tqdm.auto import tqdm

In [2]:
ratings = pd.read_parquet('/pio/scratch/1/recommender_systems/interim/Amazon/Clothing_Shoes_and_Jewelry_clean.parquet')
categories = pd.read_json('/pio/scratch/1/recommender_systems/interim/Amazon/meta_Clothing_Shoes_and_Jewelry_categories.json', lines=True)

In [3]:
# items_to_chose_from = (ratings.asin.value_counts() > 5).index

In [3]:
items_to_chose_from = np.load('/pio/scratch/1/recommender_systems/interim/ngcf/abx_test_indices.npy', allow_pickle=True)

In [5]:
def prepare_test_file(categories, path, test_set_size = 10000,
                      category_type = "category_1", weighted_sample = True,
                      min_counts = 0, items_to_chose_from = None):
    
    counts = categories[category_type].value_counts()
    counts = counts[counts > min_counts] 
    categories_grouped = categories.groupby(category_type)
    
    categories_dict = {}
    for category, items in categories_grouped:
        if items_to_chose_from is not None:
            categories_dict[category] = items.loc[items['asin'].isin(items_to_chose_from)]
        else:
            categories_dict[category] = items                                         

    test_set = []    
    for _ in tqdm(range(test_set_size)):
        
        # choosing categories for ABX
        if weighted_sample:
            category_sample = counts.sample(2, weights=counts)
        else:
            category_sample = counts.sample(2)
            
        # choosing which category to treat as positive & negative    
        positive = category_sample.index[0]
        negative = category_sample.index[1]
        if random.random() < 0.5:
            positive, negative = negative, positive
            
        # chosing items for ABX from positive & nega    
        positive_items = categories_dict[positive].sample(2).asin.values
        negative_item = categories_dict[negative].sample(1).asin.values
        
        # appending record
        line = {"A": positive_items[0],
                "B": negative_item[0],
                "X": positive_items[1],
                "category_AX": positive,
                "category_B": negative}
        test_set.append(line)
        
    with open(path, 'w') as file:    
        pd.DataFrame(test_set).to_json(file, 'records', lines = True)

In [ ]:
prepare_test_file(categories, '/pio/scratch/1/recommender_systems/interim/ABX_tests/for_ngcf.json', category_type='category_1', test_set_size = 10000, items_to_chose_from = items_to_chose_from)

  0%|          | 0/10000 [00:00<?, ?it/s]